In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import lightgbm as lgb

from hyperopt import hp, tpe, fmin, Trials
from hyperopt.pyll.base import scope

import mlflow
import mlflow.lightgbm

import warnings
warnings.filterwarnings("ignore")

import sys
import os
sys.path.append('/Users/ben/Desktop/py_proj/account_funds_prediction')
#sys.path.append('/Users/ben/Desktop/py_proj/account_funds_prediction/src')

from src.lightgbm_model_v3 import lightgbm_dev
from utils.metrics import PowerRatio

import warnings
warnings.filterwarnings("ignore")

In [2]:
sys.path

['/opt/anaconda3/envs/myenv1/lib/python310.zip',
 '/opt/anaconda3/envs/myenv1/lib/python3.10',
 '/opt/anaconda3/envs/myenv1/lib/python3.10/lib-dynload',
 '',
 '/Users/ben/Desktop/py_proj/account_funds_prediction/venv1/lib/python3.10/site-packages',
 '/Users/ben/Desktop/py_proj/account_funds_prediction']

In [3]:
os.getcwd()

'/Users/ben/Desktop/py_proj/account_funds_prediction/notebook'

## 1. Data prep

In [4]:
df = pd.read_csv('/Users/ben/Desktop/py_proj/account_funds_prediction/data/df_v1')

In [5]:
df.head()

,bal_future,bal,age,tenure,credit_score,annual_income,mtg_balance,credit_card_balance,col8,col9,...,col11,col12,col13,col14,col15,col16,col17,col18,col19,col20
0,121958,245279,87,25,781,172012,960679,3446,777,586,...,211,442,62,345,498,517,89,360,902,78
1,671155,178356,65,31,773,143327,153601,11009,248,194,...,53,946,387,843,221,424,26,940,249,383
2,131932,752233,58,13,302,161961,441341,41415,272,334,...,954,37,376,843,583,89,450,337,709,404
3,365838,895983,38,34,497,152383,554472,16728,174,722,...,96,151,402,808,198,354,452,16,648,49
4,259178,63724,82,32,786,106959,748662,1960,8,559,...,276,793,994,592,36,272,89,503,925,999


In [6]:
X = df.drop(columns=['bal'])
y = df['bal_future']

In [7]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

## 2. Model Training

### 2.1 Baseline model

In [8]:
baseline_params = {
    'max_depth': 3,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'feature_fraction': 0.5
}

search_space = {
    # Integer parameters
    'max_depth': scope.int(hp.quniform('max_depth', 3,10,1)),
    'n_estimators': scope.int(hp.quniform('n_estimators', 50,300,50)),
    'num_leaves': scope.int(hp.quniform('num_leaves', 20, 100, 5)),  
    'min_data_in_leaf': scope.int(hp.quniform('min_data_in_leaf', 5, 100, 5)),  

    # Decimal parameters
    'learning_rate': hp.uniform('learning_rate', 0.01, 1),  # Typical range for learning rate
    'feature_fraction': hp.uniform('feature_fraction', 0.3, 1.0),  # Typical range for feature_fraction
    'bagging_fraction': hp.uniform('bagging_fraction', 0.3, 1.0),  # Typical range for bagging_fraction
    'lambda_l1': hp.uniform('lambda_l1', 0, 1),  # Typical range for lambda_l1 (L1 regularization)
}


In [9]:
lgb_dev1 = lightgbm_dev(X_train=X_train, y_train=y_train, 
                   X_test=X_test, y_test=y_test,
                   baseline_params = baseline_params,
                   baseline_ind=1,
                   max_evals=2,
                   search_space=search_space)

In [10]:
lgb_dev1

In [11]:
lgb1 = lgb_dev1.run_model()

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001793 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4713
[LightGBM] [Info] Number of data points in the train set: 70000, number of used features: 20
[LightGBM] [Info] Start training from score 500775.555514
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furthe

In [12]:
lgb1

(LGBMRegressor(feature_fraction=0.5, max_depth=3, n_estimators=50),
 'e233784137a14ce4862c32e6408d1398')

In [13]:
lgb1_id = lgb1[1]
lgb1_id

'e233784137a14ce4862c32e6408d1398'

In [14]:
# Fetch the run details using MLflow API
run = mlflow.get_run(lgb1_id)

# Retrieve the metrics from the run
rmse_train = run.data.metrics.get("rmse_train", None)
rmse_test = run.data.metrics.get("rmse_test", None)
r2_train = run.data.metrics.get("r2_train", None)
r2_test = run.data.metrics.get("r2_test", None)
pr_train = run.data.metrics.get("powerratio_train", None)
pr_test = run.data.metrics.get("powerratio_test", None)

# Print the retrieved metrics
print(f"RMSE Train: {rmse_train}")
print(f"RMSE Test: {rmse_test}")
print(f"R2 Train: {r2_train}")
print(f"R2 Test: {r2_test}")
print(f"Power Ratio Train: {pr_train}")
print(f"Power Ratio Test: {pr_test}")

RMSE Train: 30129.585462629297
RMSE Test: 30185.957005566383
R2 Train: 0.989155310883988
R2 Test: 0.9890146038296852
Power Ratio Train: 0.9999343661175427
Power Ratio Test: 0.9999347809017975


### 2.2 HyperOpt (2 fits)

In [15]:
lgb_dev2 = lightgbm_dev(X_train=X_train, y_train=y_train, 
                   X_test=X_test, y_test=y_test,
                   baseline_params = baseline_params,
                   baseline_ind=0,
                   max_evals=2,
                   search_space=search_space)

In [16]:
%%time
lgb2 = lgb_dev2.run_model()

[LightGBM] [Warning] lambda_l1 is set=0.9721243016476777, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9721243016476777                                
[LightGBM] [Warning] feature_fraction is set=0.8425270791814766, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8425270791814766           
[LightGBM] [Warning] bagging_fraction is set=0.5135028962808227, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5135028962808227                  
[LightGBM] [Warning] min_data_in_leaf is set=35, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=35                                           
[LightGBM] [Warning] lambda_l1 is set=0.9721243016476777, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9721243016476777                                
[LightGBM] [Warning] feature_fraction is set=0.8425270791814766, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8425270791814766           
[LightGBM]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM]

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM]

2024/12/26 13:49:03 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during lightgbm autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 7acde082c46148dbb0f568f9e9350173. Failed operations: [MlflowException("Changing param values is not allowed. Param with key=\'learning_rate\' was already logged with value=\'0.730236549985059\' for run ID=\'7acde082c46148dbb0f568f9e9350173\'. Attempted logging new value \'0.8769880053165809\'.")]')]



[LightGBM] [Warning] lambda_l1 is set=0.2570035237866054, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2570035237866054                                
[LightGBM] [Warning] feature_fraction is set=0.4460491275969355, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4460491275969355           
[LightGBM] [Warning] bagging_fraction is set=0.8704779571454411, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8704779571454411                  
[LightGBM] [Warning] min_data_in_leaf is set=95, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=95                                           
[LightGBM] [Warning] lambda_l1 is set=0.2570035237866054, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2570035237866054                                
[LightGBM] [Warning] feature_fraction is set=0.4460491275969355, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4460491275969355           
[LightGBM]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [17]:
lgb2

(LGBMRegressor(bagging_fraction=np.float64(0.5135028962808227),
               feature_fraction=np.float64(0.8425270791814766),
               lambda_l1=np.float64(0.9721243016476777),
               learning_rate=np.float64(0.730236549985059), max_depth=5,
               min_data_in_leaf=35, n_estimators=300, num_leaves=45),
 '7acde082c46148dbb0f568f9e9350173')

In [18]:
# Fetch the run details using MLflow API
run_id = lgb2[1]
run = mlflow.get_run(run_id)

# Retrieve the metrics from the run
rmse_train = run.data.metrics.get("rmse_train", None)
rmse_test = run.data.metrics.get("rmse_test", None)
r2_train = run.data.metrics.get("r2_train", None)
r2_test = run.data.metrics.get("r2_test", None)
pr_train = run.data.metrics.get("powerratio_train", None)
pr_test = run.data.metrics.get("powerratio_test", None)

# Print the retrieved metrics
print(f"RMSE Train: {rmse_train}")
print(f"RMSE Test: {rmse_test}")
print(f"R2 Train: {r2_train}")
print(f"R2 Test: {r2_test}")
print(f"Power Ratio Train: {pr_train}")
print(f"Power Ratio Test: {pr_test}")

RMSE Train: 965.6911070256559
RMSE Test: 1754.5446681675517
R2 Train: 0.9999888594339517
R2 Test: 0.999962886263178
Power Ratio Train: 0.99999667409993
Power Ratio Test: 0.9999881765134584
